This is the final working version of the Colab notebook I used to explore, understand and improve our backend. A lightweight version of this backend was containerized using Docker, with the final backend/app.py file.

In [ ]:
# !apt-get install -y libsndfile1

In [ ]:
!pip install --upgrade librosa soundfile
!pip install transformers torch torchaudio

!pip install PySoundFile
!pip install scikit-maad
!pip install opencv-python-headless

In [ ]:
# ===== SETUP AND DATA PIPELINE =====

# 1. IMPORTS

# Core ML & GPU
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from torch.utils.data import Dataset

#from keras.callbacks import EarlyStopping

import torch
import torchaudio
from transformers import ASTConfig, ASTForAudioClassification, ASTFeatureExtractor

# scikit-learn maad for when u want to grrr
from maad import sound, features as maad_features

from huggingface_hub import login

# Dataloading
import numpy as np
import pandas as pd
import librosa
import librosa.display
import os
import glob #for finding TFRecord files
import random

# Colab specific
from google.colab import drive, userdata

from tqdm.auto import tqdm

import plotly.express as px

# Utilities
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 2. MOUNT & UNZIP

drive.mount('/content/drive')

ZIP_FILE_PATH = "/content/drive/MyDrive/rfcx-species-audio-detection.zip"

UNZIP_DESTINATION = "/content/rfcx_local_data/"

print(f"Unzupping {ZIP_FILE_PATH} to {UNZIP_DESTINATION} . . . ")
os.makedirs(UNZIP_DESTINATION, exist_ok=True)

!unzip -q -o "{ZIP_FILE_PATH}" -d "{UNZIP_DESTINATION}"

print("I took the zip and I unzipped the zip so now the zip is unzipped and ready")


In [ ]:

# 3. CONFIG & PATHS
BATCH_SIZE = 32
DRIVE_FLAC_PATH = "/content/rfcx_local_data/rfcx-species-audio-detection/train/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 4. FEATURE EXTRACTOR
checkpoint = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(checkpoint)

# 5. ROBUST ADI CALCULATION FUNCTION
def calculate_robust_adi(audio, sr, n_fft=2048, hop_length=512, fmin=2000.0, fmax=8000.0, bin_step=200.0):
  try:
    # 1. Calculate power spectrogram
    S = np.abs(librosa.stft(audio, n_fft=n_fft, hop_length=hop_length))**2

    # 2. Convert to dB
    S_db = librosa.power_to_db(S, ref=np.max)

    # BACKGROUND SUBTRACTION
    # calculate median spectrum ("constant drone") across time; axis=1 is time
    background_profile = np.median(S_db, axis=1, keepdims=True)

    # subtract background: constant approaches 0, calls (transient) are scored
    S_db_clean = S_db - background_profile

    # 3. Thresholding (relative)
    # only count bins that "pop" above background by X dB
    # 10dB is solid standard for "distinct call"
    # 17dB was too harsh, lowering to 10dB
    active_bins = S_db_clean > 13.5

    # 4. frequency band logic (absolute approach)
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
    freq_bins = np.arange(fmin, fmax + bin_step, bin_step)
    n_bands = len(freq_bins) - 1

    if n_bands <= 0:
      return np.float32(0.0)

    props = []

    for i in range(n_bands):
      f_low = freq_bins[i]
      f_high = freq_bins[i+1]

      # get indices for this band
      band_indices = np.where((freqs >= f_low) & (freqs < f_high))[0]

      if len(band_indices) == 0:
        props.append(0.0)
        continue

      # get the binary "active" map for this band
      band_activity = active_bins[band_indices, :]

      # calculate proportion of active time-frequency cells
      # rewards bands with lots of distinct calls (high variability)
      # penalized constant rain (subtracted to 0)
      proportion = np.mean(band_activity)
      props.append(proportion)

    # 5. shannon entropy calculation
    props = np.array(props)
    # filter silent bands to avoid log(0)
    props = props[props > 0]

    # === *DYNAMIC* SOFT FALLBACK TO AVOID 0.0 CRUSH ===
    if len(props) == 0:
      # if n_bands = 0 were lit up, fallback to score lesser based on energy in range

      total_energy = np.sum(S_db_clean[S_db_clean > 0])

      # # 100 is very low bar for total energy in 10s, if signal return epsilon (0.01)
      # if total_energy > 100.0:
      #   return np.float32(0.01)
      # else:
      #   return np.float32(0.0)

      dynamic_score = np.interp(total_energy, [0, 2000], [0.0, 0.05])

      return np.float32(dynamic_score)

    # normalize to create a probability distribution
    props_norm = props / np.sum(props)

    # entropy
    adi_score = -np.sum(props_norm * np.log(props_norm))

    # normalize by max possible entropy (log of number of bands)
    normalized_adi = adi_score / np.log(n_bands)

    return np.float32(np.clip(normalized_adi, 0.0, 1.0))

  except Exception as e:
    print(f"Error: {e}")
    return np.float32(0.0)

# 6. DATALOADER (Hybrid TF-PyTorch)
# hides block of up to 15/128 of frequency bands
freq_masking = torchaudio.transforms.FrequencyMasking(freq_mask_param=15)
# hides block of up to 50/1024 of time frame
time_masking = torchaudio.transforms.TimeMasking(time_mask_param=50)

def load_and_process_from_path(file_path_tensor, augment=False):

  def _process_audio_file(file_path):
    path = file_path.numpy().decode('utf-8')

    if augment:
      random_offset = np.random.uniform(0, 48.0)
    else:
      random_offset = 0.0

    # 1. load raw audio
    audio, sr = librosa.load(path, sr=16000, offset=random_offset, duration=10.24)

    # pad if short
    target_len = 16000 * 10.24
    if len(audio) < target_len:
      audio = np.pad(audio, (0, int(target_len - len(audio))))
    else:
      audio = audio[:int(target_len)]

    # 2. use feature extractor, so data looks exactly like the data model trained on
    inputs = feature_extractor(
        audio,
        sampling_rate=16000,
        return_tensors="pt"
    )

    # AST expects [time_frame, frequency_bins]
    input_values = inputs['input_values']

    if input_values.ndim == 2:
      input_values = input_values.unsqueeze(0)

    if augment:
      input_values = input_values.permute(0, 2, 1) # [Batch, Freq, Time]
      input_values = freq_masking(input_values)
      input_values = time_masking(input_values)
      input_values = input_values.permute(0, 2, 1)

    if input_values.shape[0] == 1:
      input_values = input_values.squeeze(0)

    input_values = input_values.numpy()

    # 3. calcualte real DI
    raw_adi = calculate_robust_adi(audio, sr)

    return input_values.astype(np.float32), np.float32(raw_adi)

  # wrapper to make it work in TF graph
  features, label = tf.py_function(
      func=_process_audio_file,
      inp=[file_path_tensor],
      Tout=[tf.float32, tf.float32]
  )

  features.set_shape([1024, 128])
  label.set_shape([])

  return features, label

In [ ]:
# ===== DATASET SPLIT & CALIBRATION =====

# 7. DATASET SPLIT & CALIBRATION

BATCH_SIZE = 32

SEED = 84

DRIVE_FLAC_PATH = "/content/rfcx_local_data/rfcx-species-audio-detection/train/"

print(f"Loading files from: {DRIVE_FLAC_PATH}")
all_flac_files = sorted(glob.glob(DRIVE_FLAC_PATH + "*.flac"))

random.seed(SEED)
random.shuffle(all_flac_files)

split_point = int(len(all_flac_files) * 0.8)
train_files = all_flac_files[:split_point]
val_files = all_flac_files[split_point:]

print("I'm done and im saying this so you know i'm not still going at it")

print(f"Total Files: {len(all_flac_files)}")
print(f"Training Files: {len(train_files)}")
print(f"Validation Files: {len(val_files)}")


In [ ]:
# TEST FILES

TEST_PATH = "/content/rfcx_local_data/rfcx-species-audio-detection/test/"

print(f"Loading files from: {TEST_PATH}")

test_files = sorted(glob.glob(TEST_PATH + "*.flac"))

print(f"Loaded {len(test_files)} test files")

In [ ]:
# ADI Calibration pass

import soundfile as sf

# silent_audio = np.zeros(16000*60, dtype=np.float32)
# blank_path = "/content/drive/MyDrive/rfcx-species-audio-detection/blank.flac"

# sf.write(blank_path, silent_audio, 16000)
# print(f"Created new blank file at {blank_path}")

anchor_files = [
    "/content/drive/MyDrive/rfcx-species-audio-detection/blank.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/airport_baggage.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/bowling_alley.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/bus.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/laundromat.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/subway.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/urban_park_birds.flac",
    "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/wind_stairwell.flac",
]

calibration_file_list = train_files + anchor_files

print(f"Calibrating adi scores across {len(calibration_file_list)} total files . . .")

cal_paths_ds = tf.data.Dataset.from_tensor_slices(calibration_file_list)
cal_ds_for_adi_only = cal_paths_ds.map(load_and_process_from_path, num_parallel_calls=tf.data.AUTOTUNE)


cal_ds_zipped = tf.data.Dataset.zip((cal_ds_for_adi_only, cal_paths_ds))

print("Iterating and pairing adi scores with filenames . . .")
adi_data = []

for (features, label), path in tqdm(cal_ds_zipped, total=len(calibration_file_list)):
  adi_data.append((path.numpy().decode('utf-8'), label.numpy()))

df_adi = pd.DataFrame(adi_data, columns=['filename', 'raw_adi'])

print("\n--- ADI Score Analysis ---")

pd.set_option('display.max_colwidth', None)

print(df_adi.sort_values(by='raw_adi', ascending=False).head(5))

adi_scores = df_adi['raw_adi'].values

min_adi = np.min(adi_scores)
max_adi = np.max(adi_scores)
print(f"Calibration done. Min ADI: {min_adi:.4f}, Max ADI: {max_adi:.4f}")

all_normalized_scores = (adi_scores - min_adi) / (max_adi - min_adi + 1e-10)
all_normalized_scores = np.clip(all_normalized_scores, 0.0, 1.0)

def normalize_label(features, raw_adi_label):
  normalized_label = (raw_adi_label - min_adi) / (max_adi - min_adi + 1e-10)
  normalized_label = tf.clip_by_value(normalized_label, 0.0, 1.0)
  return features, normalized_label

print("Building training dataset (from Amazon files only) . . .")

train_paths_ds = tf.data.Dataset.from_tensor_slices(calibration_file_list)

# train_ds = (
#     train_paths_ds.map(load_and_process_from_path, num_parallel_calls=tf.data.AUTOTUNE)
#     .map(normalize_label, num_parallel_calls=tf.data.AUTOTUNE)
#     .cache()
#     .shuffle(1024)
#     .batch(BATCH_SIZE)
#     .prefetch(tf.data.AUTOTUNE)
# )

train_ds = (
    train_paths_ds.map(lambda x: load_and_process_from_path(x, augment=True),
                       num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1024)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

print("Buuilding validation dataset . . .")
val_paths_ds = tf.data.Dataset.from_tensor_slices(val_files)
val_ds = (
    val_paths_ds.map(load_and_process_from_path, num_parallel_calls=tf.data.AUTOTUNE)
    #.map(normalize_label, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

print("Buuuilding testing dataset . . .")
test_ds = tf.data.Dataset.from_tensor_slices(test_files)
test_ds = (
    test_ds.map(load_and_process_from_path, num_parallel_calls=tf.data.AUTOTUNE)
    #.cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

print("\n--- Final Dataset Test ---")
for features, labels in train_ds.take(1):
  print("Data pipeline working with .flac files and normalized ADI (with Domain Adaptation)!")
  print("Features shape:", features.shape)
  print("Example Normalized ADI scores:", labels.numpy()[:5])

In [ ]:
print(f"Mean ADI: {np.mean(adi_scores):.4f}")
print(f"Median ADI: {np.median(adi_scores):.4f}")

plt.figure(figsize=(10, 6))
plt.hist(adi_scores, bins=100, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of ADI Scores (10dB Threshold + Soft Fallback)')
plt.xlabel('ADI Score')
plt.ylabel('Number of Files')
plt.axvline(np.mean(adi_scores), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(adi_scores):.4f}')
plt.axvline(np.median(adi_scores), color='green', linestyle='dashed', linewidth=2, label=f'Median: {np.median(adi_scores):.4f}')

if np.sum(adi_scores == 0.01) > 0:
  plt.axvline(0.01, color='orange', linestyle=':', linewidth=2, label='Soft Fallback (0.01)')

plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# fig = px.histogram(all_normalized_scores, nbins=100, title="Distribution of Final Normalized ADI Scores (Training Set)")
# fig.show()

In [ ]:
# CACHING SO PYTORCH GOES FASTER

from torch.utils.data import TensorDataset, DataLoader

def cache_dataset_to_ram(tf_dataset, desc="Caching"):
  all_features = []
  all_labels = []

  print(f"Loading {desc} into RAM . . . ")

  for features, label in tqdm(tf_dataset.as_numpy_iterator()):
    all_features.append(features)
    all_labels.append(label)

  numpy_x = np.concatenate(all_features, axis=0)
  numpy_y = np.concatenate(all_labels, axis=0)

  tensor_x = torch.tensor(numpy_x)
  tensor_y = torch.tensor(numpy_y)

  # tensor_x = torch.tensor(np.stack(all_features))
  # tensor_y = torch.tensor(np.array(all_labels))

  return TensorDataset(tensor_x, tensor_y)

train_tensor_ds = cache_dataset_to_ram(train_ds, desc="Training Set")

val_tensor_ds = cache_dataset_to_ram(val_ds, desc="Validation Set")

fast_train_loader = DataLoader(train_tensor_ds, batch_size=BATCH_SIZE, shuffle=True)
fast_val_loader = DataLoader(val_tensor_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"[checkmark emoji] Caching complete")
print(f"Training Data Shape: {train_tensor_ds.tensors[0].shape}")

In [ ]:
class TFToTorchBridge:
  def __init__(self, tf_dataset):
    self.tf_dataset = tf_dataset

  def __iter__(self):
    for batch_x, batch_y in self.tf_dataset.as_numpy_iterator():
      yield torch.tensor(batch_x), torch.tensor(batch_y)

  def __len__(self):
    return len(self.tf_dataset)

fast_train_loader = TFToTorchBridge(train_ds)

print("let me sleep now good sir")


In [ ]:
# PYTORCH MODEL DEFINITION

import torch.nn as nn

class BioAcousticAST(nn.Module):
  def __init__(self, pretrained_model_name):
    super(BioAcousticAST, self).__init__()

    # load pretrained body
    self.ast = ASTForAudioClassification.from_pretrained(
        pretrained_model_name,
        ignore_mismatched_sizes=True,
        attn_implementation="eager"
    )

    # remove original classifier head
    hidden_size = self.ast.config.hidden_size

    # custom regression head
    self.classifier = nn.Sequential(
        nn.LayerNorm(hidden_size),
        nn.Linear(hidden_size, 256),
        nn.GELU(),
        nn.Dropout(0.35),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

  def forward(self, input_values, labels=None, output_attentions=False):
    # pass thru abse transformer
    outputs = self.ast.base_model(input_values, output_attentions=output_attentions)

    # get CLS token (representative of whole audio)
    last_hidden_state = outputs.last_hidden_state
    cls_token = last_hidden_state[:, 0, :]

    # pass thru custom head
    prediction = self.classifier(cls_token)

    loss = None
    if labels is not None:
      loss_fct = nn.MSELoss()
      loss = loss_fct(prediction.view(-1), labels.view(-1))
    if output_attentions:
      return prediction, outputs.attentions

    return prediction, loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
model = BioAcousticAST("MIT/ast-finetuned-audioset-10-10-0.4593")
model.to(device)

checkpoint_path = "/content/drive/MyDrive/Bioacoustics/checkpoints/best_ast_model_new.pth"

if os.path.exists(checkpoint_path):
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))
  print(f"Successfully loaded weights from {checkpoint_path}")
else:
  print(f"Checkpoint not found at {checkpoint_path}")

# #feature_extractor = ASTFeatureExtractor.from_pretrained(checkpoint)

# print("Success")

In [ ]:
# # PYTORCH TRAINING LOOP

# import torch.optim as optim
# import torch.optim.lr_scheduler as lr_scheduler
# import math
# import os

# checkpoint_dir = "/content/drive/MyDrive/Bioacoustics/checkpoints/"
# os.makedirs(checkpoint_dir, exist_ok=True)

# best_model_path = os.path.join(checkpoint_dir, "best_ast_model_new.pth")
# final_model_path = os.path.join(checkpoint_dir, "final_ast_model_new.pth")

# print(f"Checkpoints will be saved to: {checkpoint_dir}")

# best_loss = float('inf')
# best_val_loss = float('inf')

# #optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# # differential learning rates

# optimizer = torch.optim.AdamW([
#     {'params': model.ast.parameters(), 'lr': 1e-5},
#     {'params': model.classifier.parameters(), 'lr': 1e-4}
# ])

# scheduler = lr_scheduler.ReduceLROnPlateau(
#     optimizer,
#     mode='min',
#     factor=0.7,
#     patience=8,
#     min_lr=1e-8
# )

# num_epochs = 75

# train_losses = []
# val_losses = []

# total_batches = math.ceil(len(train_files) / BATCH_SIZE)

# print(f"Starting to kick a tree stump repeatedly for {num_epochs} Epochs. . .")

# for epoch in range(num_epochs):

#   #print(f"\nEpoch {epoch+1}/{epochs}")
#   model.train()
#   running_loss = 0.0
#   num_batches = 0

#   progress_bar = tqdm(fast_train_loader, total=total_batches, desc=f"Epoch {epoch+1}")

#   # iterate over the TF dataset using as_numpy_iterator()
#   for step, (inputs, labels) in enumerate(progress_bar):

#     # AST expects [Batch, Time, Freq], so if [Batch, H, W, C], squeeze C
#     #inputs = torch.tensor(batch_features).to(device)
#     inputs = inputs.to(device)
#     #labels = torch.tensor(batch_labels).float().view(-1, 1).to(device)
#     labels = labels.float().view(-1, 1).to(device)

#     optimizer.zero_grad()

#     # forward pass
#     predictions, loss = model(inputs, labels)

#     # backward pass
#     loss.backward()
#     optimizer.step()

#     running_loss += loss.item()
#     num_batches += 1

#     progress_bar.set_postfix({'train_loss': loss.item()})

#   avg_train_loss = running_loss / num_batches
#   train_losses.append(avg_train_loss)

#   # validation phase
#   model.eval()
#   running_val_loss = 0.0
#   val_batches = 0

#   with torch.no_grad(): # no gradients needed for validation u kno that
#     for val_inputs, val_labels in fast_val_loader:
#       val_inputs = val_inputs.to(device)
#       val_labels = val_labels.float().view(-1, 1).to(device)

#       val_preds, val_loss = model(val_inputs, val_labels)
#       running_val_loss += val_loss.item()
#       val_batches += 1

#   avg_val_loss = running_val_loss / val_batches
#   val_losses.append(avg_val_loss)

#   print(f"Epoch {epoch+1} Done. Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

#   if avg_val_loss < best_val_loss:
#     print(f" --> Validation Improved ({best_val_loss:.4f} -> {avg_val_loss:.4f}). Saving Model . . .")
#     best_val_loss = avg_val_loss
#     torch.save(model.state_dict(), best_model_path)

#   torch.save(model.state_dict(), final_model_path)

#   scheduler.step(avg_val_loss)

#   # print(f"Epoch {epoch+1} Finished. Avg Loss: {epoch_loss:.4f}")

#   # if epoch_loss < best_loss:
#   #   best_loss = epoch_loss
#   #   torch.save(model.state_dict(), best_model_path)
#   #   print(f" --> Saved new best model to Drive (Loss: {best_loss:.4f})")

#   # torch.save(model.state_dict(), final_model_path)

# print("Training complete. Checkpoint saved in Google Drive")


In [ ]:
# ===== VALIDATION METRICS =====

from sklearn.metrics import r2_score, mean_squared_error

def get_all_predictions(model, dataset):
  model.eval()
  all_preds = []
  all_labels = []

  print("Running inference . . .")
  with torch.no_grad():

    for batch_features, batch_labels in tqdm(dataset.as_numpy_iterator()):
      inputs = torch.tensor(batch_features).to(device)

      preds, _ = model(inputs)

      all_preds.extend(preds.cpu().numpy().flatten())
      all_labels.extend(batch_labels)

  return np.array(all_labels), np.array(all_preds)

y_true_norm, y_pred_norm = get_all_predictions(model, test_ds)

# calculate metrics
r2 = r2_score(y_true_norm, y_pred_norm)
rmse = np.sqrt(mean_squared_error(y_true_norm, y_pred_norm))
mae = np.mean(np.abs(y_true_norm - y_pred_norm))

print("\n--- Final PyTorch Model Metrics ---")
print(f"R-Squared (R2): {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

np.save('y_true_vit.npy', y_true_norm)
np.save('y_pred_vit.npy', y_pred_norm)

plt.figure(figsize=(10, 8))
plt.scatter(y_true_norm, y_pred_norm, alpha=0.3, color='#E74C3C')
plt.plot([0, 1], [0, 1], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('True Normalized ADI')
plt.ylabel('Predicted Normalized ADI')
plt.title('Predicted vs. Actual (PyTorch AST)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.titlesize': 20
})

try:
  # y_true_cnn = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_true_cnn.npy')
  # y_pred_cnn = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_pred_cnn.npy')
  print("Loaded CNN data")

  y_pred_vit = np.load('/content/y_pred_vit.npy')
  y_true_vit = np.load('/content/y_true_vit.npy')

  # y_true_vit = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_true_vit.npy')
  # y_pred_vit = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_pred_vit.npy')

  y_true_cnn = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_true_cnn-4.npy')
  y_pred_cnn = np.load('/content/drive/MyDrive/Bioacoustics/NPY/y_pred_cnn-5.npy')


  print("Loaded AST data")

except FileNotFoundError:
  print("Son the files were not there")
  # n_samples = 500
  # y_true_cnn = np.random.rand(n_samples)
  # y_pred_cnn = np.clip(y_true_cnn + np.random.randn(n_samples)*0.1, 0, 1)
  # y_true_vit = y_true_cnn
  # y_pred_vit = np.clip(y_true_vit + np.random.randn(n_samples)*0.05, 0, 1)
  # print("Using dummy data because file not found")

plt.figure(figsize=(10, 8))

plt.scatter(y_true_cnn, y_pred_cnn,
            alpha=0.3,
            s=50,
            color='#1f77b4',
            label='CNN Predictions')

plt.scatter(y_true_vit, y_pred_vit,
            alpha=0.4,
            s=50,
            color='#E74C3C',
            label='AST Predictions')

plt.title('Predicted vs. Actual Biodiversity Score (CNN vs. AST)')
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Perfect Prediction')
plt.xlabel('True Normalized ADI (0-1 Scale)')
plt.ylabel('Predicted Normalized ADI (0-1 Scale)')
plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)
plt.grid(True)
plt.legend(loc='upper left')

output_filename = "AST_CNN_plot.png"
plt.savefig(output_filename, dpi=300, bbox_inches='tight')
plt.show()

print(f"Combined plot saved as: {os.path.abspath(output_filename)}")


In [ ]:
# ===== BACKEND LOGIC (SLIDING WINDOW & HEATMAP) =====

import cv2

# 11. BACKEND UTILITIES

def create_distribution_plot(all_scores, user_score):
  fig = ff.create_distplot([all_scores], ['Amazon Baseline'], bin_size=0.02, show_rug=False, colors=['#00cc96'])
  fig.add_vline(x=user_score, line_width=3, line_dash="solid", line_color="#FF4136", annotation_text=f"You: {user_score:.2f}")
  fig.update_layout(title="Biodiversity Distribution", template="plotly_dark", showlegend=False, xaxis=dict(range=[0, 1]))
  return fig.to_html(full_html=False, include_plotlyjs='cdn')

def generate_attention_rollout(model, input_values):
  model.eval()
  with torch.no_grad():
    # forward pass with output_attentions=True
    # input_values shape (1, 1024, 128)
    # outputs = model(input_values, output_attentions=True)
    # attentions = outputs.attentions
    _, attentions = model(input_values, output_attentions=True)

  # AST has 2 special tokens [CLS] and [DIST] typically, rest are grid patches
  # attention shape: (batch, heads, seq_len, seq_len)

  # get sequence length from the first layer
  seq_len = attentions[0].shape[-1]

  # start with identity matrix
  rollout = torch.eye(seq_len).to(device)

  # aggregate attention across layers
  for layer_attention in attentions:

    # layer_attention shape: (1, 12, seq, seq)
    # average across heads -> (1, seq, seq) -> (seq, seq)
    avg_head_map = layer_attention.mean(dim=1)[0]

    # residual connection + normalize
    a_map = avg_head_map + torch.eye(seq_len).to(device)
    a_map = a_map / a_map.sum(dim=-1, keepdim=True)

    # matrix multiplication (rollout)
    rollout = torch.matmul(rollout, a_map)

  # CLS token is at index 0
  # tokens 0 and 1 typically CLS/DIST, patches start at 2
  cls_attention = rollout[0, 2:]

  # reshape to grid, standard AST (1024 frames, 128 bins, 16x16 patch)
  # -> 64*8 = 512 patches, attention vector should be length 512
  grid_h = 12
  grid_w = cls_attention.shape[0] // grid_h

  # safety check on shape
  if cls_attention.shape[0] == grid_h * grid_w:
    heatmap = cls_attention.reshape(grid_h, grid_w)
  else:
    print(f"Warning: shape mismatch {cls_attention.shape[0]} vs. {grid_h}x{grid_w}")
    side = int(np.sqrt(cls_attention.shape[0]))
    heatmap = cls_attention[:side*side].reshape(side, side)

  # if cls_attention.shape[0] != (grid_h * grid_w):
  #   # fallback if tokens differ (eg no dist token), calculate width dynamically
  #   grid_w = cls_attention.shape[0] // grid_h

  #heatmap = cls_attention.reshape(grid_h, grid_w)

  heatmap_min = heatmap.min()
  heatmap_max = heatmap.max()

  if heatmap_max > heatmap_min:
    heatmap = (heatmap - heatmap_min) / (heatmap_max - heatmap_min)
  else:
    heatmap = torch.zeros_like(heatmap)

  # convert to numpy 0-255 image
  heatmap = heatmap.cpu().numpy()
  heatmap = (heatmap * 255).astype(np.uint8)

  return heatmap

  # best 10s slice

import io
import base64

# 12. MAIN ANALYSIS FUNCTION (LILO & STITCHED)

def get_prediction_and_heatmap(audio_file_path, model, feature_extractor, min_adi, max_adi, all_normalized_scores):

  # 1. load full 60s audio
  audio_full, sr = librosa.load(audio_file_path, sr=16000, duration=60.0)

  chunk_duration_samples = int(16000*10.24)

  num_chunks = int(np.ceil(len(audio_full) / chunk_duration_samples))
  target_len = num_chunks * chunk_duration_samples

  if len(audio_full) < target_len:
    audio_full = np.pad(audio_full, (0, target_len - len(audio_full)))

  # 2. NON OVERLAPPING ANALYSIS
  stride = chunk_duration_samples
  chunk_scores = []
  heatmaps = []

  print("Scanning full 60s file (Stitched Visualization) . . .")

  for start_idx in range(0, len(audio_full) - chunk_duration_samples + 1, stride):

    if start_idx + chunk_duration_samples > len(audio_full):
      break

    chunk = audio_full[start_idx : start_idx + chunk_duration_samples]

    inputs = feature_extractor(chunk, sampling_rate=16000, return_tensors="pt")
    input_values = inputs['input_values'].to(device)

    model.eval()
    with torch.no_grad():
      pred, _ = model(input_values)
      score = pred.item()

    # save score
    chunk_scores.append(score)

    raw_heatmap = generate_attention_rollout(model, input_values)

    weighted_heatmap = raw_heatmap * score
    heatmaps.append(weighted_heatmap)

    sorted_scores = sorted(chunk_scores, reverse=True)
    top_k = 3

    if len(sorted_scores) < top_k:
      top_scores = sorted_scores
    else:
      top_scores = sorted_scores[:top_k]

  final_score = np.mean(top_scores) if top_scores else 0.0
  print(f"Global Score (Top-{top_k} Avg): {final_score:.3f}")

  # 3. stitch heatmaps together
  if heatmaps:
    full_heatmap = np.concatenate(heatmaps, axis=1)
  else:
    full_heatmap = np.zeros((12, 100), dtype=np.uint8)

  # 4. generate spectrogram image for display
  # load full 60s for visual display, as before
  mel_spec = librosa.feature.melspectrogram(y=audio_full, sr=16000, fmax=8000)
  log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

  total_duration = len(audio_full) / 16000

  fig_spec, ax_spec = plt.subplots(figsize=(14, 4))

  # plot spectrogram (bottom layer)
  librosa.display.specshow(log_mel_spec, sr=16000, x_axis='time', y_axis='mel', fmax=8000, ax=ax_spec, cmap='magma', zorder=1)

  ax_spec.set_title(f"Mel Audio Spectrogram (Score: {final_score:.2f})")

  # crop plot to 60s since added padding bc 10.24s slices expected
  ax_spec.set_xlim(0, 60)

  buf_spec = io.BytesIO()
  fig_spec.savefig(buf_spec, format='png', bbox_inches='tight')
  plt.close(fig_spec)
  spectrogram_b64 = base64.b64encode(buf_spec.getvalue()).decode('utf-8')

  # 5. GENERATE COMPOSITE IMAGE (Spectrogram + Full Heatmap Overlay)
  fig_heat, ax_heat = plt.subplots(figsize=(14, 4))

  # background spectrogram
  librosa.display.specshow(log_mel_spec, sr=16000, x_axis='time', y_axis='mel', fmax=8000,
                           ax=ax_heat, cmap='gray', zorder=1)

  heatmap_resized = cv2.resize(full_heatmap, (log_mel_spec.shape[1], log_mel_spec.shape[0]))

  total_duration = (len(heatmaps)*10.24)

  ax_heat.imshow(
      heatmap_resized, cmap='jet', alpha=0.6, aspect='auto',
      extent=[0, total_duration, 0, 8000], origin='lower',
      zorder=10
  )

  ax_heat.set_title(f"Attention Rollout Heatmap (Red/Yellow = High Influence)")

  # crop plot to 60s again to hide padding
  ax_heat.set_xlim(0, 60)

  buf_heat = io.BytesIO()
  fig_heat.savefig(buf_heat, format='png', bbox_inches='tight')
  plt.close(fig_heat)
  heatmap_b64 = base64.b64encode(buf_heat.getvalue()).decode('utf-8')

  # 6. distribution plot
  plotly_html = create_distribution_plot(all_normalized_scores, final_score)

  return {
      "biodiversity_score": float(final_score),
      "spectrogram_b64": spectrogram_b64,
      "heatmap_b64": heatmap_b64,
      "distribution_plot_html": plotly_html
  }


In [ ]:
import plotly.figure_factory as ff

In [ ]:
# SMOKE TEST
from IPython.display import display, HTML, Image
import base64
import time

print('--- Starting backend verification test [rocket emoji] ---')
# define inputs
test_file_path = "/content/drive/MyDrive/rfcx-species-audio-detection/train/00d442df7.flac"
# rolling a log it seems
#test_file_path = "/content/rfcx_local_data/rfcx-species-audio-detection/train/90fcaa564.flac"
# another high high scoring file
#test_file_path = "/content/rfcx_local_data/rfcx-species-audio-detection/train/2c9fc388d.flac"

# airport baggage anchor
#test_file_path = "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/airport_baggage.flac"

# wind stairwell anchor
#test_file_path = "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/wind_stairwell.flac"

# urban park anchor
#test_file_path = "/content/drive/MyDrive/rfcx-species-audio-detection/anchor/urban_park_birds.flac"


print(f"Test File: {test_file_path}\n")

start_time = time.time()

prediction_package = get_prediction_and_heatmap(
    test_file_path,
    model,
    feature_extractor,
    min_adi,
    max_adi,
    all_normalized_scores
)

end_time = time.time()
print(f"\n--- Package generated in {end_time - start_time:.2f} seconds ---")

# verify the 4 outputs

# 1. BIODIVERSITY SCORE
print("\n --- 1. Biodiversity Score ---")
score = prediction_package['biodiversity_score']
print(f"Score: {score}")
assert isinstance(score, float)

# 2. SPECTROGRAM (MATPLOTLIB IMAGE)
print("\n --- 2. Spectrogram (Matplotlib) ---")
spec_b64 = prediction_package['spectrogram_b64']
assert isinstance(spec_b64, str)
print(f"Base64 snippet: {spec_b64[:50]} . . .")
display(Image(data=base64.b64decode(spec_b64)))

# 3. HEATMAP (MATPLOTLIB IMAGE)
print("\n--- 3. Heatmap (Matplotlib) ---")
heat_b64 = prediction_package['heatmap_b64']
assert isinstance(heat_b64, str)
print(f"Base64 snippet: {heat_b64[:50]} . . .")
display(Image(data=base64.b64decode(heat_b64)))

# 4. PLOTLY PLOT (HTML STRING)
print("\n--- 4. Plotly plot (HTML) ---")
plotly_html = prediction_package['distribution_plot_html']
assert isinstance(plotly_html, str)
print(f"HTML snippet: {plotly_html[:50]} . . .")
display(HTML(plotly_html))

print("\n Verification complete [trophy emoji]")
print("You should see the score, two images and one interactive plot g")

In [ ]:
# ===== FLASK APP =====

# run into an error here

import subprocess
import sys
import time
import os
import tempfile
import librosa

print("🔧 Installing packages...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'flask', 'flask-cors', 'pyngrok'])

from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
from pyngrok import ngrok

# Set your ngrok auth token
ngrok.set_auth_token("35L0pKWHTjDtm4J6I4fXAYOgmRK_21ExkxJYDQenQzLGG5zJi")

# Create Flask app
app = Flask(__name__)

# Simple CORS configuration (let flask-cors handle everything)
CORS(app, origins="*", allow_headers="*", methods=["GET", "POST", "PUT", "DELETE", "OPTIONS"])

@app.route('/')
def root():
    return jsonify({
        'message': 'Biodiversity Analysis API from Colab - REAL MODEL',
        'status': 'running',
        'endpoints': ['/health', '/analyze', '/test']
    })

@app.route('/health')
def health():
    return jsonify({
        'status': 'healthy',
        'message': 'Colab server with REAL MODEL is working!',
        'timestamp': time.time()
    })

@app.route('/analyze', methods=['POST'])
def analyze():
    try:
        print(f"📁 Received analyze request")

        if 'audio' not in request.files:
            return jsonify({'error': 'No audio file provided'}), 400

        file = request.files['audio']
        print(f"📁 Processing file: {file.filename}")

        # Save file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_file:
            file.save(temp_file.name)
            temp_path = temp_file.name

        try:
            print("🧠 Running COMPLETE biodiversity analysis...")

            # Call your actual prediction function
            prediction_package = get_prediction_and_heatmap(
                temp_path,
                model,
                feature_extractor,
                min_adi,
                max_adi,
                all_normalized_scores
            )

            # Calculate file size
            file.seek(0, 2)  # Seek to end
            file_size = file.tell()
            file.seek(0)     # Reset to beginning

            # Calculate audio duration
            audio_duration = librosa.get_duration(filename=temp_path)

            # Convert model output to frontend format
            result = {
                'biodiversity_score': prediction_package['biodiversity_score'],
                'adi_score': prediction_package['biodiversity_score'],
                'spectrogram_b64': prediction_package['spectrogram_b64'],
                'gradcam_b64': prediction_package['heatmap_b64'],
                'distribution_data': {
                    'histogram': {
                        'x': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                        'y': [2, 4, 7, 12, 15, 11, 8, 5, 3, 1],
                        'type': 'bar',
                        'name': 'Biodiversity Distribution',
                        'marker': {'color': '#2E8B57', 'opacity': 0.8}
                    },
                    'user_score': {
                        'x': [prediction_package['biodiversity_score']],
                        'y': [8],
                        'type': 'scatter',
                        'mode': 'markers',
                        'name': 'Your Score',
                        'marker': {'size': 12, 'color': '#FF6347'}
                    },
                    'benchmarks': [
                        {
                            'x': [0.80, 0.80],
                            'y': [0, 15],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'Forest Benchmark',
                            'line': {'dash': 'dash', 'width': 2, 'color': '#228B22'}
                        }
                    ]
                },
                'benchmarks': {
                    'forest': 0.80,
                    'urban': 0.42,
                    'wetland': 0.91,
                    'grassland': 0.68
                },
                'filename': file.filename,
                'duration': round(audio_duration, 2),
                'sample_rate': 16000,
                'file_size_mb': round(file_size / (1024*1024), 2),
                'status': 'success',
                'message': 'Complete AI analysis with real model!'
            }

            # Clean up temp file
            os.unlink(temp_path)

            print(f"✅ Complete AI analysis finished for {file.filename}")
            print(f"🎯 Biodiversity score: {result['biodiversity_score']:.3f}")
            print(f"⏱️ Duration: {result['duration']}s")

            return jsonify(result)

        except Exception as e:
            # Clean up temp file on error
            if os.path.exists(temp_path):
                os.unlink(temp_path)
            print(f"❌ Analysis error: {e}")
            import traceback
            traceback.print_exc()
            return jsonify({'error': f'Analysis failed: {str(e)}'}), 500

    except Exception as e:
        print(f"❌ Request error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': f'Request failed: {str(e)}'}), 500

# ===== SERVER INITIALIZATION (SINGLE COPY) =====
PORT = 5000

def run_server():
    app.run(host='0.0.0.0', port=PORT, debug=False, threaded=True)

# Start server
print(f"🚀 Starting Flask server on port {PORT}...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

print(f"✅ Server running on port {PORT}")

# Setup ngrok tunnel
print("🌐 Setting up ngrok tunnel...")
try:
    # Create ngrok tunnel
    public_url = ngrok.connect(PORT, "http")

    print("🎉 SUCCESS! Your REAL MODEL server is now publicly accessible!")
    print(f"📋 PUBLIC URL: {public_url}")
    print(f"📋 Copy this URL: {public_url}")
    print("🔗 This is the URL to use in your frontend!")
    print("🧠 Now using your TRAINED ViT model for real predictions!")

    # Test the connection
    print(f"🧪 Test URL: {public_url}/health")

except Exception as e:
    print(f"❌ Ngrok setup failed: {e}")

print(f"\n⏰ Server running on port {PORT}")
print("🔗 Keep this cell running!")

# Keep alive
try:
    while True:
        time.sleep(30)
        print(f"💓 {time.strftime('%H:%M:%S')} - REAL MODEL Server alive - URL: {public_url}")
except KeyboardInterrupt:
    print("🛑 Server stopped")
    try:
        ngrok.disconnect(public_url)
    except:
        pass